In [113]:
#Uncomment to install the following packages
#!pip install googletrans==4.0.0-rc1 
#!pip install unidecode
#!pip install langdetect

from googletrans import Translator
from unidecode import unidecode
from langdetect import detect

#Imports
import os

#Initialised translator
translator = Translator()

#Declares a function that checks if a string is fully english
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

#Asks the user for a path to do the operations on - path is hardcoded for now
path = "C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy" #input("File Path: ")
#If there is no backslash in the path, the bath is not in the windows user path and the path doesn't exist, a new path is requested
while "\\" not in path and ":\\Users" not in path and os.path.isdir(path):
    print("File Path Invalid")
    path = input("Please Select A Different File Path: ")

#Gets rid of trailing backslashes or any file names attached to the end of the path
if path.split("\\")[-1] == "" or "." in path.split("\\")[-1]:
    path = path.rsplit("\\", 1)[0]

#Arrays used to hold file names, their translations and their path
fullFilePaths = [] #unstranslated file paths
rootOfPaths = [] #transalted file paths

#Used to hold untranslated and trasnlated chunks of file name text
textToTrasnlateChunks = []
textToTrasnlate = ""
translatedText = ""

#Recursively goes through all directories and finds files within them
for root, directories, files in os.walk(path):
    for file in files:
        #Each file name is appended to a string with a new line
        #When that string reaches 3000 characters, it is stored as a chunk of text in the array
        #The API seems to only allow about 5000 chars to be sent and translated per call
        #Since google as a API call limit and translated text can have more chars than the original text
        #chuncks of 3000 texts is sent per API call and then separated later. This makes translation faster and more efficient.
        #Note - I haven't tested to see if it's fine for the original text to have more than 5000 char after translation
        #Just palyin' it safe!
        if len(textToTrasnlate) >= 3000:
            textToTrasnlateChunks.append(textToTrasnlate) #Note - the last training new line is cut off when stored - so a new line should be added when the chunks are recombined
            textToTrasnlate = ""

        #Each file name is seperated by a new line
        textToTrasnlate += file.rsplit(".", 1)[0] + os.linesep

        fullFilePaths.append(os.path.join(root, file)) #The full file for each path is stored here
        rootOfPaths.append(root) #Adds the path to each file's directory - the transalted file name is added to it later

#Appends the remainder of the file names to the chunk array - so partial chunks/the remaining file names that didn't form a full chunk can be translated
textToTrasnlateChunks.append(textToTrasnlate.rsplit(os.linesep, 1)[0])
textToTrasnlate = ""

#Translates each chunk of text from japanese, korean, cantonese and then mandarin to english
translatedChunk = ""
for chunk in textToTrasnlateChunks:
    translatedChunk = translator.translate(chunk).text
    #translatedChunk = translator.translate(translatedChunk).text
    #translatedChunk = translator.translate(translatedChunk).text
    
    #translatedChunk = translator.translate(chunk, src='ja', dest='en').text
    #translatedChunk = translator.translate(translatedChunk, src='ko', dest='en').text
    #translatedChunk = translator.translate(translatedChunk, src='zh-cn', dest='en').text
    #translatedChunk = translator.translate(translatedChunk, src='zh-tw', dest='en').text
    translatedText += translatedChunk + os.linesep
    
translatedFileNameList = translatedText.splitlines()
duplicates = []

#Checks for duplicates in list of file name translation
for fileName in translatedFileNameList:
    if translatedFileNameList.count(fileName) > 1:
        #Makes note the file names of files with duplicates
        duplicates.append(fileName)

#Makes list distinct
duplicates = list(set(duplicates))
        
#Handles duplicates if any are found
while duplicates:
    #Character to add in the suffix "-[]" which is appended to duplicates
    duplicateSuffixChar = "I"
    duplicateSuffix = "-[" + duplicateSuffixChar + "]"
    
    #For each duplicate file name, the given file name is given a suffix that attempts to distinguise it
    for dup in duplicates:
        #Keeps track of the amount of "I"s we want in the "-[]" suffix 
        suffixRepeater = 0
        #Keeps track of the index of the translated filename being inspected
        i = 0
        for filename in translatedFileNameList:
            #If a translated filename is a duplicate, a suffix is added
            if filename == dup:
                #Adds I to suffix
                for i in range(suffixRepeater):
                    duplicateSuffix = duplicateSuffix[:2] + "I" + duplicateSuffix[2:]
                suffixRepeater += 1 #Insures the suffix will have an extra "I" when the next duplicate is found
                translatedFileNameList[i] = filename + duplicateSuffix #Adds suffix to translated file
            i += 1
    
    duplicates = []
    #Checks for the duplication handler created any duplicates that need further handling
    for fileName in translatedFileNameList:
        if translatedFileNameList.count(fileName) > 1:
            duplicates.append(fileName)

#For debugging
#print("Number of lines of translated text: " + str(len(translatedText.split(os.linesep)[:-1])))
#print("Number of file paths: " + str(len(rootOfPaths)))
#print("last: " + str(translatedText.split(os.linesep)[-2]))

for i in range(len(translatedFileNameList)):
    rootOfPaths[i] = os.path.join(rootOfPaths[i], (translatedFileNameList[i] + ".mp3"))
    os.rename(fullFilePaths[i], rootOfPaths[i].replace("\"", "'").replace("?", "？")[:250])

print(translatedFileNameList)
#print(rootOfPaths)

['C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\Drama CD Lucky Star + Drama Complete Disk + Moe Drill + True Lucky Star Moe Drill.mp3', 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\Drama CD VOL5.mp3', 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\DRAMA CDS From Tomorrow.mp3', "C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\Emue Mu! Drama CD 'Esubi!'.mp3", 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\Servant X Service X Working !! Drama CD.mp3', 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\The Demon King!.mp3', 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\Treasure Voice？ [Hyakuni One Neck] Hyakemono of Rie Tanaka.mp3', 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\[DRAMA CD] [for Work] My GirffRIEND AND HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER HER.mp3', 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\[DRAMA CD] [for Work] SOMA SOMA 11 Volum

In [132]:
#Uncomment to install the following packages
#!pip install googletrans==4.0.0-rc1 
#!pip install unidecode
#!pip install langdetect

from googletrans import Translator
from unidecode import unidecode
from langdetect import detect

#Imports
import os

#Initialised translator
translator = Translator()

#Declares a function that checks if a string is fully english
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

#Asks the user for a path to do the operations on - path is hardcoded for now
path = "C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy" #input("File Path: ")
#If there is no backslash in the path, the bath is not in the windows user path and the path doesn't exist, a new path is requested
while "\\" not in path and ":\\Users" not in path and os.path.isdir(path):
    print("File Path Invalid")
    path = input("Please Select A Different File Path: ")

#Gets rid of trailing backslashes or any file names attached to the end of the path
if path.split("\\")[-1] == "" or "." in path.split("\\")[-1]:
    path = path.rsplit("\\", 1)[0]

#Arrays used to hold file names, their translations and their path
fullFilePaths = [] #unstranslated file paths
rootOfPaths = [] #transalted file paths

#Used to hold untranslated and trasnlated chunks of file name text
textToTranslateChunks = []
textToTranslate = ""
translatedText = ""

#Language Specific chunk holders
textToTranslate_en = ""
textToTranslate_vi = ""
textToTranslate_ja = ""
textToTranslate_ko = ""
textToTranslate_zh_cn = ""
textToTranslate_zh_tw = ""
textToTranslate_CatchAll = ""

#Langauge specific translation holders
translatedText_en = ""
translatedText_vi = ""
translatedText_ja = ""
translatedText_ko = ""
translatedText_zh_cn = ""
translatedText_zh_tw = ""
translatedText_CatchAll = ""

#Language specific chunks
textToTranslateChunks_en = []
textToTranslateChunks_vi = []
textToTranslateChunks_ja = []
textToTranslateChunks_ko = []
textToTranslateChunks_zh_cn = []
textToTranslateChunks_zh_tw = []
textToTranslateChunks_CatchAll = []

#Reassembly positions
textToTranslateChunks_en_pos = []
textToTranslateChunks_vi_pos = []
textToTranslateChunks_ja_pos = []
textToTranslateChunks_ko_pos = []
textToTranslateChunks_zh_cn_pos = []
textToTranslateChunks_zh_tw_pos = []
textToTranslateChunks_CatchAll_pos = []

i = 0
#Recursively goes through all directories and finds files within them
for root, directories, files in os.walk(path):
    for file in files:
        i += 1
        if(detect(file.rsplit(".", 1)[0]) == "ja"):
            if len(textToTranslate_ja) >= 2000:
                textToTranslateChunks_ja.append(textToTranslate_ja)
                textToTranslate_ja = ""
            textToTranslate_ja += file.rsplit(".", 1)[0] + os.linesep
            textToTranslateChunks_ja_pos.append(i)
            
        elif(detect(file.rsplit(".", 1)[0]) == "en"):
            if len(textToTranslate_en) >= 2000:
                textToTranslateChunks_en.append(textToTranslate_en)
                textToTranslate_en = ""
            textToTranslate_en += file.rsplit(".", 1)[0] + os.linesep
            textToTranslateChunks_en_pos.append(i)
            
        elif(detect(file.rsplit(".", 1)[0]) == "vi"):
            if len(textToTranslate_vi) >= 2000:
                textToTranslateChunks_vi.append(textToTranslate_vi)
                textToTranslate_vi = ""
            textToTranslate_vi += file.rsplit(".", 1)[0] + os.linesep
            textToTranslateChunks_vi_pos.append(i)
            
        elif(detect(file.rsplit(".", 1)[0]) == "ko"):
            if len(textToTranslate_ko) >= 2000:
                textToTranslateChunks_ko.append(textToTranslate_ko)
                textToTranslate_ko = ""
            textToTranslate_ko += file.rsplit(".", 1)[0] + os.linesep
            textToTranslateChunks_ko_pos.append(i)
            
        elif(detect(file.rsplit(".", 1)[0]) == "zh-cn"):
            if len(textToTranslate_zh_cn) >= 2000:
                textToTranslateChunks_zh_cn.append(textToTranslate_zh_cn)
                textToTranslate_zh_cn = ""
            textToTranslate_zh_cn += file.rsplit(".", 1)[0] + os.linesep
            textToTranslateChunks_zh_cn_pos.append(i)
            
        elif(detect(file.rsplit(".", 1)[0]) == "zh-tw"):
            if len(textToTranslate_zh_tw) >= 2000:
                textToTranslateChunks_zh_tw.append(textToTranslate_zh_tw)
                textToTranslate_zh_tw = ""
            textToTranslate_zh_tw += file.rsplit(".", 1)[0] + os.linesep
            textToTranslateChunks_zh_tw_pos.append(i)
            
        else:
            if len(textToTranslateChunks_CatchAll_pos) >= 2000:
                textToTranslateChunks_CatchAll.append(textToTranslate)
                textToTranslateChunks_CatchAll_pos = ""
            textToTranslate_CatchAll += file.rsplit(".", 1)[0] + os.linesep
            textToTranslateChunks_CatchAll_pos.append(i)

        if len(textToTranslate) >= 2000:
            textToTranslateChunks.append(textToTranslate) #Note - the last training new line is cut off when stored - so a new line should be added when the chunks are recombined
            textToTranslate = ""

        #Each file name is seperated by a new line
        textToTranslate += file.rsplit(".", 1)[0] + os.linesep

        fullFilePaths.append(os.path.join(root, file)) #The full file for each path is stored here
        rootOfPaths.append(root) #Adds the path to each file's directory - the transalted file name is added to it later

#Appends the remainder of the file names to the chunk array - so partial chunks/the remaining file names that didn't form a full chunk can be translated
textToTranslateChunks.append(textToTranslate.rsplit(os.linesep, 1)[0])
textToTranslate = ""

textToTranslateChunks_en.append(textToTranslate_en.rsplit(os.linesep, 1)[0])
textToTranslateChunks_vi.append(textToTranslate_vi.rsplit(os.linesep, 1)[0])
textToTranslateChunks_ja.append(textToTranslate_ja.rsplit(os.linesep, 1)[0])
textToTranslateChunks_ko.append(textToTranslate_ko.rsplit(os.linesep, 1)[0])
textToTranslateChunks_zh_cn.append(textToTranslate_zh_cn.rsplit(os.linesep, 1)[0])
textToTranslateChunks_zh_tw.append(textToTranslate_zh_tw.rsplit(os.linesep, 1)[0])
textToTranslateChunks_CatchAll.append(textToTranslate_CatchAll.rsplit(os.linesep, 1)[0])

textToTranslate_en = ""
textToTranslate_vi = ""
textToTranslate_ja = ""
textToTranslate_ko = ""
textToTranslate_zh_cn = ""
textToTranslate_zh_tw = ""
textToTranslate_CatchAll = ""

print("en")
for chunk in textToTranslateChunks_en:
    print(chunk)
    if len(chunk) == 0:
        break
    translatedText_en += chunk + os.linesep
    
print("vi")
for chunk in textToTranslateChunks_vi:
    print(chunk)
    if len(chunk) == 0:
        break
    translatedText_vi += (translator.translate(chunk).text if (len(translator.translate(chunk).text) > 0) else chunk) + os.linesep

print("ja")  
for chunk in textToTranslateChunks_ja:
    print(chunk)
    if len(chunk) == 0:
        break
    translatedText_ja += (translator.translate(chunk).text if (len(translator.translate(chunk).text) > 0) else chunk) + os.linesep

print("ko")  
for chunk in textToTranslateChunks_ko:
    print(chunk)
    if len(chunk) == 0:
        break
    translatedText_ko += (translator.translate(chunk).text if (len(translator.translate(chunk).text) > 0) else chunk) + os.linesep

print("cn") 
for chunk in textToTranslateChunks_zh_cn:
    print(chunk)
    if len(chunk) == 0:
        break
    translatedText_zh_cn += (translator.translate(chunk).text if (len(translator.translate(chunk).text) > 0) else chunk) + os.linesep

print("tw")  
for chunk in textToTranslateChunks_zh_tw:
    print(chunk)
    if len(chunk) == 0:
        break
    translatedText_zh_tw += (translator.translate(chunk).text if (len(translator.translate(chunk).text) > 0) else chunk) + os.linesep

print("catch all")
for chunk in textToTranslateChunks_CatchAll:
    print(chunk)
    if len(chunk) == 0:
        break
    translatedText_CatchAll += chunk + os.linesep

translatedFileNameList2 = translatedText_ja.splitlines() + translatedText_ko.splitlines() + translatedText_zh_cn.splitlines() + translatedText_zh_tw.splitlines() + translatedText_CatchAll.splitlines() + translatedText_en.splitlines() + translatedText_vi.splitlines()
translatedFileNameList2Indexs = textToTranslateChunks_ja_pos + textToTranslateChunks_ko_pos + textToTranslateChunks_zh_cn_pos + textToTranslateChunks_zh_tw_pos + textToTranslateChunks_CatchAll_pos + textToTranslateChunks_en_pos + textToTranslateChunks_vi_pos
finalTranslatedList = [x for _, x in sorted(zip(translatedFileNameList2Indexs, translatedFileNameList2))]

#Translates each chunk of text from japanese, korean, cantonese and then mandarin to english
translatedChunk = ""
for chunk in textToTranslateChunks:
    translatedChunk = translator.translate(chunk).text
    #translatedChunk = translator.translate(translatedChunk).text
    #translatedChunk = translator.translate(translatedChunk).text
    
    #translatedChunk = translator.translate(chunk, src='ja', dest='en').text
    #translatedChunk = translator.translate(translatedChunk, src='ko', dest='en').text
    #translatedChunk = translator.translate(translatedChunk, src='zh-cn', dest='en').text
    #translatedChunk = translator.translate(translatedChunk, src='zh-tw', dest='en').text
    translatedText += translatedChunk + os.linesep
    
translatedFileNameList = translatedText.splitlines()
duplicates = []

translatedFileNameList = finalTranslatedList

#Checks for duplicates in list of file name translation
for fileName in translatedFileNameList:
    if translatedFileNameList.count(fileName) > 1:
        #Makes note the file names of files with duplicates
        duplicates.append(fileName)

#Makes list distinct
duplicates = list(set(duplicates))
        
#Handles duplicates if any are found
while duplicates:
    #Character to add in the suffix "-[]" which is appended to duplicates
    duplicateSuffixChar = "I"
    duplicateSuffix = "-[" + duplicateSuffixChar + "]"
    
    #For each duplicate file name, the given file name is given a suffix that attempts to distinguise it
    for dup in duplicates:
        #Keeps track of the amount of "I"s we want in the "-[]" suffix 
        suffixRepeater = 0
        #Keeps track of the index of the translated filename being inspected
        i = 0
        for filename in translatedFileNameList:
            #If a translated filename is a duplicate, a suffix is added
            if filename == dup:
                #Adds I to suffix
                for i in range(suffixRepeater):
                    duplicateSuffix = duplicateSuffix[:2] + "I" + duplicateSuffix[2:]
                suffixRepeater += 1 #Insures the suffix will have an extra "I" when the next duplicate is found
                translatedFileNameList[i] = filename + duplicateSuffix #Adds suffix to translated file
            i += 1
    
    duplicates = []
    #Checks for the duplication handler created any duplicates that need further handling
    for fileName in translatedFileNameList:
        if translatedFileNameList.count(fileName) > 1:
            duplicates.append(fileName)

#For debugging
#print("Number of lines of translated text: " + str(len(translatedText.split(os.linesep)[:-1])))
#print("Number of file paths: " + str(len(rootOfPaths)))
#print("last: " + str(translatedText.split(os.linesep)[-2]))

for i in range(len(translatedFileNameList)):
    rootOfPaths[i] = os.path.join(rootOfPaths[i], (translatedFileNameList[i] + ".mp3"))
    os.rename(fullFilePaths[i], rootOfPaths[i].replace("\"", "'").replace("?", "？").replace(":", "：")[:250])

print(translatedFileNameList)
#print(rootOfPaths)

en
Tomorrow Night's Pixie Arc - We Never Learn BOKUBEN Part 1
Tomorrow Night's Pixie Arc - We Never Learn BOKUBEN Part 2
Tomorrow Night's Pixie Arc - We Never Learn BOKUBEN Part 3
Tomorrow Night's Pixie Arc - We Never Learn BOKUBEN Part 4
Tomorrow Night's Pixie Arc - We Never Learn BOKUBEN Part 5
Tomorrow Night's Pixie Arc - We Never Learn BOKUBEN Part 6
Tomorrow Night's Pixie Arc - We Never Learn BOKUBEN Part 7 (Final)
Arc Sleeping Beauty of the Literary Forest Part 8 (Final)
Sleeping Beauty of the Literary Forest Arc Part 1
Sleeping Beauty of the Literary Forest Arc Part 2
Sleeping Beauty of the Literary Forest Arc Part 3
Sleeping Beauty of the Literary Forest Arc Part 4
Sleeping Beauty of the Literary Forest Arc Part 5
Sleeping Beauty of the Literary Forest Arc Part 6
Sleeping Beauty of the Literary Forest Arc Part 7
The Queen of Thin Ice Arc - We Never Learn BOKUBEN Part 1
The Queen of Thin Ice Arc - We Never Learn BOKUBEN Part 10
The Queen of Thin Ice Arc - We Never Learn BOKUBEN 

美坂栞ストーリー (2) - 再会、冬の約束、白い羽の少女。
美坂栞ストーリー (3) - 憧れ、零れ落ちてゆくもの。
美坂栞ストーリー (5) - お気に入りの場所、同じ景色、舞踏会。
美坂栞ストーリー (6) - 紡がれない言葉、友達、姉妹ということ。
美坂栞ストーリー (7) - 雪。
美坂栞ストーリー (8) - お願い。
美坂栞ストーリー (9) - 夢の季節。
未確認で進行形 11巻特典 ドラマCD
未確認で進行形 ドラマCD 「つまり転ばないように乗ればいんですね」
未確認で進行形 ドラマCD「媚茶ですが…」
【夏目友人帳】 ドラマCD ２編まとめ！ 【アニメ‘ｓ jp】
【ドラマCD風】ニューゲーム！！ 「メイド喫茶がいいと言ったんだよ（歓迎会） , 他」
【ドラマCD風】ニューゲーム！！「おつかい、初任給」
【ドラマＣＤ】【作業用】ニセコイ同梱第9巻 内山昴輝 東山奈央
【ニセコイ】 ドラマCD ３編まとめ！ 【アニメ‘ｓ jp】
サウンドドラマ プリンセスコネクト! ReDive PRICONNE CHARACTER SONG11・12・14・15のドラマまとめ
サウンドドラマ プリンセスコネクト! ReDive PRICONNE CHARACTER SONG1～5のドラマまとめ
サウンドドラマ プリンセスコネクト! ReDive PRICONNE CHARACTER SONG6～10のドラマまとめ
GA文库《哥布林杀手》10卷特装版付广播剧 ゴブリンスレイヤー ドラマＣＤ マン･ウィズ･ザ･ミッション ガール･ウィズ･ザ･パッション 1 2
GA文库《哥布林杀手》10卷特装版付广播剧 ゴブリンスレイヤー ドラマＣＤ マン･ウィズ･ザ･ミッション ガール･ウィズ･ザ･パッション 2 2
PSYCHO-PASS 0 dramaCD（上） 01 むかしばなしをはじめようか
PSYCHO-PASS 0 dramaCD（上） 02 おひめさまとおうじさま
PSYCHO-PASS 0 dramaCD（上） 03 わるいまほうつかいののろい
xxxHOLiC オリジナルドラマCD 「シミヌキ」
xxxHOLiC 継 ドラマCD
[广播剧]魔女之旅 DRAMA CD GAノベル『魔女の旅々17』ドラマCD試聴PV①
[广播剧]魔女之旅 DRAMA CD GAノ

【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 人生的迷道
【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 诱拐
【广播剧】笨蛋、测验、召唤兽《僕と子と召喚獣》
【熟肉】《朋友的妹妹只喜欢烦我》第4卷特装版付广播剧 彩羽のウザ絡み耐久ボイス集
【生肉 广播剧】三坪房间的侵略者！？33卷 特典CD 01 きっかけは信号待ち
【神様になった日】 麻枝 准書き下ろしドラマCD「ラッパーになった日 -2nd BATTLE-」
【神様になった日】 麻枝 准書き下ろしドラマCD「ラッパーになった日」
【響け！ユーフォニアム】 ドラマCD アニメとは一味違う、オリジナルエピソード！ 【アニメＣＤ】
【魔法科高校の劣等生】 ドラマCD デートをするお兄様を見た深雪が.
あの夏で待ってる ドラマCD
あんハピ♪ サウンドドラマCD
お兄ちゃんだけど愛さえあれば関係ないよねっドラマcd
からかい上手の高木さん Vol.2.4.6 Blu-ray&DVD 初回生産限定版 オーディオドラマCD①.②.③
からかい上手の高木さん2 Vol.1~Vol.6 Blu-ray＆DVD 初回生産限定版 タカギーデジタルサラウンドサウンドシステム ver.(バイノーラルサラウンド)【音声のみ】
この美術部には問題がある！ ドラマCD
これゾンドラマCD
さくら荘のペットな彼女ドラマCD視聴編
すかすか ドラマCD
はぢがーる
ひぐらし ドラマCD罪滅し編 竜宮レナ 一世一代の頑張り物語終盤
ひぐらしのなく頃に ドラマCD 暇潰し編 「そして彼らは立ち上がる」
ひぐらしのなく頃に粋 紗都子ＶＳ雲雀１３
ひぐらしのなく頃に解 ドラマCD 祭囃し編 「君を助けに来た！！」
ひぐらしのなく頃に解 ドラマCD 罪滅し編 レナの過去 その２
ひぐらしのなく頃に解 ドラマCD 罪滅し編 レナの過去
ひぐらしのなく頃に解 ドラマCD罪滅し編 オヤシロさまの降臨
ひぐらしのなく頃に解 ドラマCD罪滅し編 スクラップ帖
ひぐらしのなく頃に解 ドラマCD罪滅し編 レナを追う影
ひぐらしのなく頃に解 ドラマCD罪滅し編 北条鉄平の惨い最期
ひぐらしのなく頃に解 ドラマCD罪滅し編 圭一ＶＳレナ その2
ひぐらしのなく頃に解 ドラマCD罪滅し編 圭一ＶＳレナ その１
ひぐらしのなく頃に解 ドラマ

【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 暴风雨中也一往无前！
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 暴风雨和庆典中的宝来岛
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 更加吃惊 第二关！
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 洞窟的秘密
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 片头曲 祈祷之钟
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 第一 主持人登场
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 第三 主持人登场
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 第二 主持人登场
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 第五 主持人登场
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 第四 主持人登场
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 终曲
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 被毁坏的仓库和令人毛骨悚然的先行组
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 谁烧了？ 第一关
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 谜语名人是，绕口令名人？
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 谜语挑战 第三关
【广播剧】犬夜叉drama 《暴风雨和庆典中的宝来岛（岚と祭りの宝来岛）》 闪耀！第一回 犬夜叉奖！！
【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 又一个便当盒
【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 回到战国时代！
【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 堆满盘子的黑山
【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 很快天亮
【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 意外的出场者
【广播剧】犬夜叉drama 《红と白の歌合戦! ～犬夜叉版～》 戈薇和珊瑚


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\DRAMA CD vol5.mp3' -> "C：\\Users\\Rayha\\Desktop\\Voice Dramas - Copy\\[Shrimp] Inuyasha DRAMA 'Seven people who were waiting for the ground of the ground' (Seven people who were waiting for the ground) '-[IIIIIII].mp3"